In [1]:
from utils import (
    get_dataset_size_from_model_size, calculate_total_steps, calculate_total_flops, calculate_flops_per_step,
    calculate_num_h100s_per_step, calculate_total_time_to_train_a_model,
    compute_minimum_latency_between_clusters, calculate_total_minimum_comm_latency_to_train_a_model,
    get_time_per_step, get_training_cost
)
from utils import (
    convert_to_petaflops, convert_to_exaflops, convert_seconds_to_days,
    convert_to_xt_format, convert_to_million_format, convert_to_billion_format, convert_seconds_to_years
)
from constants import UTILIZED_BFLOAT16_FLOPS, H100_COST_PER_HOUR, H100_COST_PER_GPU
import pandas as pd
import numpy as np

In [2]:
target_model_sizes = [100*10**9, 500*10**9, 1000*10**9, 10000*10**9, 100000*10**9]

#### Critical batch size

In [3]:
from utils import get_critical_batch_size, convert_to_million_format

In [4]:
critical_batch_sizes = [get_critical_batch_size(x) * 4096 for x in target_model_sizes]

In [5]:
df_gbs = pd.DataFrame({
    'Model Size (B params)': [convert_to_xt_format(x) for x in target_model_sizes],
    'Critical Batch Size (tokens)': [f'{x/1e6:1f}M' for x in critical_batch_sizes]
})

In [6]:
df_gbs

,Model Size (B params),Critical Batch Size (tokens)
0,0.1T,4.740704M
1,0.5T,8.025570M
2,1.0T,10.067991M
3,10.0T,21.381728M
4,100.0T,45.409090M


#### Compute

In [7]:
# 100b to 100T
global_batch_sizes = [x*10**6 for x in [2, 10, 40]]

In [8]:
time_per_step = 1 # the total time of a fwd, and bwd pass

In [9]:
dataframes_compute = []

for global_batch_size in global_batch_sizes:
    data_compute = {
        "Model Size (Params)": [],
        "Dataset Size (Tokens)": [],
        "Global Batch Size": [],
        "Total Steps": [],
        "Total FLOPs": [],
        "FLOPs per Step": [],
        "H100 GPUs Needed": [],
        "Total H100s cost": [],
        "Total Training Time without grad accum": [],
        "Total Training Time with 10 grad accum": [],
        "Total Training Time with 100 grad accum": [],
        "Total Training Time with 1000 grad accum": []
    }
    
    for model_size in target_model_sizes:
        dataset_size = get_dataset_size_from_model_size(model_size)
        total_steps = calculate_total_steps(model_size, global_batch_size)
        total_flops = calculate_total_flops(model_size)
        flops_per_step = calculate_flops_per_step(model_size, global_batch_size)
        h100s_per_step = calculate_num_h100s_per_step(model_size, global_batch_size, UTILIZED_BFLOAT16_FLOPS)

        time_per_step = get_time_per_step(model_size, global_batch_size, UTILIZED_BFLOAT16_FLOPS)
        total_time = calculate_total_time_to_train_a_model(model_size, global_batch_size, time_per_step)
        total_training_cost = get_training_cost(model_size, global_batch_size, UTILIZED_BFLOAT16_FLOPS, H100_COST_PER_HOUR)
        
        data_compute["Model Size (Params)"].append(convert_to_xt_format(model_size))
        data_compute["Dataset Size (Tokens)"].append(convert_to_xt_format(dataset_size))
        data_compute["Global Batch Size"].append(f'{global_batch_size/1e6}M')
        data_compute["Total Steps"].append("{:,}".format(total_steps))
        data_compute["Total FLOPs"].append(convert_to_exaflops(total_flops))
        data_compute["FLOPs per Step"].append(convert_to_petaflops(flops_per_step))
        data_compute["H100 GPUs Needed"].append(h100s_per_step)
        # data_compute["Total H100s cost"].append(convert_to_billion_format(h100s_per_step * H100_COST_PER_GPU))
        data_compute["Total H100s cost"].append(convert_to_million_format(total_training_cost))
        data_compute["Total Training Time without grad accum"].append(convert_seconds_to_days(total_time))
        data_compute["Total Training Time with 10 grad accum"].append(f"{convert_seconds_to_years(total_time*10)} - {h100s_per_step/10} gpus")
        data_compute["Total Training Time with 100 grad accum"].append(f"{convert_seconds_to_years(total_time*100)} - {h100s_per_step/100} gpus")
        data_compute["Total Training Time with 1000 grad accum"].append(f"{convert_seconds_to_years(total_time*1000)} - {h100s_per_step/1000} gpus")
    
    df = pd.DataFrame(data_compute)
    # Add batch size information
    # df['Global Batch Size'] = f'{global_batch_size/1e6}M'
    dataframes_compute.append(df)

final_df_compute = pd.DataFrame()
for i, df in enumerate(dataframes_compute):
    final_df_compute = pd.concat([final_df_compute, df])

In [10]:
final_df_compute

,Model Size (Params),Dataset Size (Tokens),Global Batch Size,Total Steps,Total FLOPs,FLOPs per Step,H100 GPUs Needed,Total H100s cost,Total Training Time without grad accum,Total Training Time with 10 grad accum,Total Training Time with 100 grad accum,Total Training Time with 1000 grad accum
0,0.1T,2.0T,2.0M,"1,000,000","1,200,000.0 EFLOPs","1,200.0 PFLOPs",2206.0,1.2m,11.6 days,0.32 years - 220.6 gpus,3.17 years - 22.06 gpus,31.69 years - 2.206 gpus
1,0.5T,10.0T,2.0M,"5,000,000","30,000,000.0 EFLOPs","6,000.0 PFLOPs",11030.0,30.6m,57.9 days,1.58 years - 1103.0 gpus,15.84 years - 110.3 gpus,158.45 years - 11.03 gpus
2,1.0T,20.0T,2.0M,"10,000,000","120,000,000.0 EFLOPs","12,000.0 PFLOPs",22060.0,122.6m,115.7 days,3.17 years - 2206.0 gpus,31.69 years - 220.6 gpus,316.89 years - 22.06 gpus
3,10.0T,200.0T,2.0M,"100,000,000","12,000,000,000.0 EFLOPs","120,000.0 PFLOPs",220608.0,"12,256.0m",1157.4 days,31.69 years - 22060.8 gpus,316.88 years - 2206.08 gpus,3168.82 years - 220.608 gpus
4,100.0T,2000.0T,2.0M,"1,000,000,000","1,200,000,000,000.0 EFLOPs","1,200,000.0 PFLOPs",2206085.0,"1,225,602.8m",11574.1 days,316.88 years - 220608.5 gpus,3168.81 years - 22060.85 gpus,31688.09 years - 2206.085 gpus
0,0.1T,2.0T,10.0M,"200,000","1,200,000.0 EFLOPs","6,000.0 PFLOPs",11030.0,1.2m,2.3 days,0.06 years - 1103.0 gpus,0.63 years - 110.3 gpus,6.34 years - 11.03 gpus
1,0.5T,10.0T,10.0M,"1,000,000","30,000,000.0 EFLOPs","30,000.0 PFLOPs",55152.0,30.6m,11.6 days,0.32 years - 5515.2 gpus,3.17 years - 551.52 gpus,31.69 years - 55.152 gpus
2,1.0T,20.0T,10.0M,"2,000,000","120,000,000.0 EFLOPs","60,000.0 PFLOPs",110304.0,122.6m,23.1 days,0.63 years - 11030.4 gpus,6.34 years - 1103.04 gpus,63.38 years - 110.304 gpus
3,10.0T,200.0T,10.0M,"20,000,000","12,000,000,000.0 EFLOPs","600,000.0 PFLOPs",1103042.0,"12,256.0m",231.5 days,6.34 years - 110304.2 gpus,63.38 years - 11030.42 gpus,633.76 years - 1103.042 gpus
4,100.0T,2000.0T,10.0M,"200,000,000","1,200,000,000,000.0 EFLOPs","6,000,000.0 PFLOPs",11030425.0,"1,225,602.8m",2314.8 days,63.38 years - 1103042.5 gpus,633.76 years - 110304.25 gpus,6337.62 years - 11030.425 gpus


##### Communication time of data parallelism

In [11]:
from constants import FP8_BYTES, BFLOAT16_BYTES
from utils import convert_bytes_to_terabytes
from utils import calculate_comm_time_given_comm_volume, convert_bytes_to_gigabytes
from constants import NVLINK_MAX_TOTAL_BANDWIDTH

Assume that fwd+bwd pass of a single replicas takes 1 seconds

In [12]:
# comm_bandwidths = [0.5*1024**3, 1*1024**3, 4*1024**3] # bytes/sec
# comm_bandwidths = [40*1024**3, NVLINK_MAX_TOTAL_BANDWIDTH] # bytes/sec
comm_bandwidths = [40*1024**3] # bytes/sec
cluster_sizes = [1024, 10240, 102400]

data_mem = {
    "Model Size (Params)": [],
    "Global batch size": [],
    # "Number of datacenters": [],
    "Total bfloat16 gradient storage": [],
    "Total fp8 gradient storage": [],
    "Bandwidth": [],
    "Total communication time in bfloat16 - comm/compute ratio": [],
    "Total communication time in fp8 - comm/compute ratio": [],
    "Total GPU idle cost for bfloat16 comm": [],
    "Total GPU idle cost for fp8 comm": [],
    "DiLoCo's total communication time in bfloat16 (500 inner steps) - comm/compute ratio": []
}
# for cluster_size in cluster_sizes:
for bandwidth in comm_bandwidths:
    for global_batch_size in global_batch_sizes:
        for model_size in target_model_sizes:
            total_steps = calculate_total_steps(model_size, global_batch_size)
            time_per_step = get_time_per_step(model_size, global_batch_size, UTILIZED_BFLOAT16_FLOPS)
            total_time = calculate_total_time_to_train_a_model(model_size, global_batch_size, time_per_step)
            
            # h100s_per_step = calculate_num_h100s_per_step(model_size, global_batch_size, UTILIZED_BFLOAT16_FLOPS)
            # num_clusters = h100s_per_step // cluster_size

            bfloat16_grad_comm_volume = model_size * BFLOAT16_BYTES
            fp8_grad_comm_volume = model_size * FP8_BYTES
            
            bfloat16_total_comm_time = calculate_comm_time_given_comm_volume(bfloat16_grad_comm_volume, bandwidth) * total_steps
            fp8_total_comm_time = calculate_comm_time_given_comm_volume(fp8_grad_comm_volume, bandwidth) * total_steps
            bfloat16_diloco_total_comm_time = calculate_comm_time_given_comm_volume(fp8_grad_comm_volume, bandwidth) * (total_steps / 500)

            bfloat16_comm_compute_ratio = (bfloat16_total_comm_time / (total_time + bfloat16_total_comm_time)) * 100
            fp8_comm_compute_ratio = (fp8_total_comm_time / (total_time + fp8_total_comm_time)) * 100
            bfloat16_diloco_comm_compute_ratio = (bfloat16_diloco_total_comm_time / (total_time + bfloat16_diloco_total_comm_time)) * 100
            
            bfloat16_total_gpu_idle_cost_comm = ((bfloat16_total_comm_time * h100s_per_step) / (60*60)) / H100_COST_PER_HOUR
            fp8_total_gpu_idle_cost_comm = ((fp8_total_comm_time * h100s_per_step) / (60*60)) / H100_COST_PER_HOUR
            
            data_mem["Model Size (Params)"].append(convert_to_xt_format(model_size))
            data_mem["Global batch size"].append(f'{global_batch_size/1e6}M')
            # data_mem["Number of datacenters"].append(num_clusters)
            data_mem["Total bfloat16 gradient storage"].append(convert_bytes_to_terabytes(model_size * BFLOAT16_BYTES))
            data_mem["Total fp8 gradient storage"].append(convert_bytes_to_terabytes(model_size * FP8_BYTES))
            data_mem["Bandwidth"].append(f"{convert_bytes_to_gigabytes(bandwidth)}/s")
            data_mem["Total communication time in bfloat16 - comm/compute ratio"].append(f"{convert_seconds_to_days(bfloat16_total_comm_time)} / {convert_seconds_to_years(bfloat16_total_comm_time)} - {bfloat16_comm_compute_ratio:.2f}%")
            data_mem["Total communication time in fp8 - comm/compute ratio"].append(f"{convert_seconds_to_days(bfloat16_total_comm_time)} / {convert_seconds_to_years(fp8_total_comm_time)} - {fp8_comm_compute_ratio:.2f}%")
            data_mem["Total GPU idle cost for bfloat16 comm"].append(convert_to_billion_format(bfloat16_total_gpu_idle_cost_comm))
            data_mem["Total GPU idle cost for fp8 comm"].append(convert_to_billion_format(fp8_total_gpu_idle_cost_comm))
            data_mem["DiLoCo's total communication time in bfloat16 (500 inner steps) - comm/compute ratio"].append(f"{convert_seconds_to_days(bfloat16_diloco_total_comm_time)}  - {bfloat16_diloco_comm_compute_ratio:.2f}%")
    
    df_mem = pd.DataFrame(data_mem)

Add cluster size

In [13]:
df_mem

,Model Size (Params),Global batch size,Total bfloat16 gradient storage,Total fp8 gradient storage,Bandwidth,Total communication time in bfloat16 - comm/compute ratio,Total communication time in fp8 - comm/compute ratio,Total GPU idle cost for bfloat16 comm,Total GPU idle cost for fp8 comm,DiLoCo's total communication time in bfloat16 (500 inner steps) - comm/compute ratio
0,0.1T,2.0M,0.200 TB,0.100 TB,42.950 GB/s,53.9 days / 0.15 years - 82.32%,53.9 days / 0.07 years - 69.95%,28.54B,14.27B,0.1 days - 0.46%
1,0.5T,2.0M,1.000 TB,0.500 TB,42.950 GB/s,1347.4 days / 3.69 years - 95.88%,1347.4 days / 1.84 years - 92.09%,713.39B,356.70B,1.3 days - 2.28%
2,1.0T,2.0M,2.000 TB,1.000 TB,42.950 GB/s,5389.6 days / 14.76 years - 97.90%,5389.6 days / 7.38 years - 95.88%,"2,853.58B","1,426.79B",5.4 days - 4.45%
3,10.0T,2.0M,20.000 TB,10.000 TB,42.950 GB/s,538959.8 days / 1475.59 years - 99.79%,538959.8 days / 737.80 years - 99.57%,"285,357.90B","142,678.95B",539.0 days - 31.77%
4,100.0T,2.0M,200.000 TB,100.000 TB,42.950 GB/s,53895982.3 days / 147559.16 years - 99.98%,53895982.3 days / 73779.58 years - 99.96%,"28,535,789.65B","14,267,894.83B",53896.0 days - 82.32%
5,0.1T,10.0M,0.200 TB,0.100 TB,42.950 GB/s,10.8 days / 0.03 years - 82.32%,10.8 days / 0.01 years - 69.95%,5.71B,2.85B,0.0 days - 0.46%
6,0.5T,10.0M,1.000 TB,0.500 TB,42.950 GB/s,269.5 days / 0.74 years - 95.88%,269.5 days / 0.37 years - 92.09%,142.68B,71.34B,0.3 days - 2.28%
7,1.0T,10.0M,2.000 TB,1.000 TB,42.950 GB/s,1077.9 days / 2.95 years - 97.90%,1077.9 days / 1.48 years - 95.88%,570.72B,285.36B,1.1 days - 4.45%
8,10.0T,10.0M,20.000 TB,10.000 TB,42.950 GB/s,107792.0 days / 295.12 years - 99.79%,107792.0 days / 147.56 years - 99.57%,"57,071.58B","28,535.79B",107.8 days - 31.77%
9,100.0T,10.0M,200.000 TB,100.000 TB,42.950 GB/s,10779196.5 days / 29511.83 years - 99.98%,10779196.5 days / 14755.92 years - 99.96%,"5,707,157.93B","2,853,578.97B",10779.2 days - 82.32%


#### Communication latency (theoretical minimum)

Assumptions on communication
- No limit on banwidth
- Achieve speed of light
- Clostest surface distance between two points on the earth surface (assume you don't dig a crazy hole to go a straight line)

In [14]:
minimum_latency_between_jz_and_jc = compute_minimum_latency_between_clusters("JEAN_ZAY", "JOLIOT_CURIE")
minimum_latency_between_jz_and_ec = compute_minimum_latency_between_clusters("JEAN_ZAY", "EL_CAPITAN")

In [15]:
dataframes_comm = []

for global_batch_size in global_batch_sizes:
    data_comm = {
        "Model Size (Params)": [],
        "Dataset Size (Tokens)": [],
        "Global Batch Size": [],
        
        "Total minimum communication latency between JZ and JC": [],
        "Total GPU idle time for minimum comm between JZ and JC": [],
        "GPU idle cost during JZ-JC minimum communication latency": [],
        
        "Total minimum communication latency between JZ and EC": [],
        "Total GPU idle time for minimum comm between JZ and EC": [],
        "GPU idle cost during JZ-EC minimum communication latency": [],
    }
    
    for model_size in target_model_sizes:
        dataset_size = get_dataset_size_from_model_size(model_size)
        # Append to dictionary
        data_comm["Model Size (Params)"].append(convert_to_xt_format(model_size))
        data_comm["Dataset Size (Tokens)"].append(convert_to_xt_format(dataset_size))
        data_comm["Global Batch Size"].append(f'{global_batch_size/1e6}M')
        
        data_comm["Total minimum communication latency between JZ and JC"].append(calculate_total_minimum_comm_latency_to_train_a_model(model_size, global_batch_size, minimum_latency_between_jz_and_jc))
        data_comm["Total GPU idle time for minimum comm between JZ and JC"].append(convert_seconds_to_days(calculate_total_minimum_comm_latency_to_train_a_model(model_size, global_batch_size, minimum_latency_between_jz_and_jc) * h100s_per_step))
        data_comm["GPU idle cost during JZ-JC minimum communication latency"].append(convert_to_million_format((calculate_total_minimum_comm_latency_to_train_a_model(model_size, global_batch_size, minimum_latency_between_jz_and_jc) / (60 * 60)) * h100s_per_step * H100_COST_PER_HOUR))
        
        data_comm["Total minimum communication latency between JZ and EC"].append(calculate_total_minimum_comm_latency_to_train_a_model(model_size, global_batch_size, minimum_latency_between_jz_and_ec))
        data_comm["Total GPU idle time for minimum comm between JZ and EC"].append(convert_seconds_to_years(calculate_total_minimum_comm_latency_to_train_a_model(model_size, global_batch_size, minimum_latency_between_jz_and_ec) * h100s_per_step))
        data_comm["GPU idle cost during JZ-EC minimum communication latency"].append(convert_to_million_format((calculate_total_minimum_comm_latency_to_train_a_model(model_size, global_batch_size, minimum_latency_between_jz_and_ec) / (60 * 60)) * h100s_per_step * H100_COST_PER_HOUR))
    
    # Convert to DataFrame
    df_comm = pd.DataFrame(data_comm)
    # df_comm['Global Batch Size'] = f'{global_batch_size/1e6}M'
    dataframes_comm.append(df_comm)

final_df_comm = pd.DataFrame()
for i, df in enumerate(dataframes_comm):
    final_df_comm = pd.concat([final_df_comm, df])

In [16]:
final_df_comm

,Model Size (Params),Dataset Size (Tokens),Global Batch Size,Total minimum communication latency between JZ and JC,Total GPU idle time for minimum comm between JZ and JC,GPU idle cost during JZ-JC minimum communication latency,Total minimum communication latency between JZ and EC,Total GPU idle time for minimum comm between JZ and EC,GPU idle cost during JZ-EC minimum communication latency
0,0.1T,2.0T,2.0M,1.278270,652.8 days,0.0m,29.790336,41.65 years,0.7m
1,0.5T,10.0T,2.0M,6.391352,3263.9 days,0.2m,148.951681,208.25 years,3.7m
2,1.0T,20.0T,2.0M,12.782705,6527.7 days,0.3m,297.903362,416.51 years,7.3m
3,10.0T,200.0T,2.0M,127.827046,65277.2 days,3.1m,2979.033618,4165.08 years,73.0m
4,100.0T,2000.0T,2.0M,1278.270459,652771.6 days,31.3m,29790.336181,41650.83 years,730.2m
0,0.1T,2.0T,10.0M,0.255654,130.6 days,0.0m,5.958067,8.33 years,0.1m
1,0.5T,10.0T,10.0M,1.278270,652.8 days,0.0m,29.790336,41.65 years,0.7m
2,1.0T,20.0T,10.0M,2.556541,1305.5 days,0.1m,59.580672,83.30 years,1.5m
3,10.0T,200.0T,10.0M,25.565409,13055.4 days,0.6m,595.806724,833.02 years,14.6m
4,100.0T,2000.0T,10.0M,255.654092,130554.3 days,6.3m,5958.067236,8330.17 years,146.0m


#### Electricity

In [17]:
from constants import TOTAL_H100_WATT
from utils import convert_watts_to_megawatts, convert_watts_to_terawatts, calculate_electricity_consumption_of_an_h100

In [18]:
dataframes_elec = []

for global_batch_size in global_batch_sizes:
    data_elec = {
        "Model Size (Params)": [],
        "Dataset Size (Tokens)": [],
        "Global batch size": [],
        "Number of GPUs": [],
        "Total electricity per step (without grad accum)": [],
        "Total electricity for the entire training (without grad accum)": []
    }
    
    for model_size in target_model_sizes:
        dataset_size = get_dataset_size_from_model_size(model_size)
        h100s_per_step = calculate_num_h100s_per_step(model_size, global_batch_size, UTILIZED_BFLOAT16_FLOPS)
        time_per_step = get_time_per_step(model_size, global_batch_size, UTILIZED_BFLOAT16_FLOPS)
        total_time = calculate_total_time_to_train_a_model(model_size, global_batch_size, time_per_step)
        total_electricity_consumption = calculate_electricity_consumption_of_an_h100(TOTAL_H100_WATT, total_time) * h100s_per_step
        
        data_elec["Model Size (Params)"].append(convert_to_xt_format(model_size))
        data_elec["Dataset Size (Tokens)"].append(convert_to_xt_format(dataset_size))
        data_elec["Global batch size"].append(f'{global_batch_size/1e6}M')
        data_elec["Number of GPUs"].append(h100s_per_step)
        data_elec["Total electricity per step (without grad accum)"].append(convert_watts_to_megawatts(h100s_per_step * TOTAL_H100_WATT))
        data_elec["Total electricity for the entire training (without grad accum)"].append(f"{convert_watts_to_terawatts(total_electricity_consumption)}")
    
    df = pd.DataFrame(data_elec)
    # df['Global Batch Size'] = f'{global_batch_size/1e6}M'
    dataframes_elec.append(df)

final_df_elec = pd.DataFrame()
for i, df in enumerate(dataframes_elec):
    final_df_elec = pd.concat([final_df_elec, df])

In [19]:
# my calculation closes to 100k gpu cluster's electricity: https://semianalysis.com/2024/06/17/100000-h100-clusters-power-network/#power-challenges

In [20]:
final_df_elec

,Model Size (Params),Dataset Size (Tokens),Global batch size,Number of GPUs,Total electricity per step (without grad accum),Total electricity for the entire training (without grad accum)
0,0.1T,2.0T,2.0M,2206.0,2.813 MW,2.813 TW
1,0.5T,10.0T,2.0M,11030.0,14.063 MW,70.319 TW
2,1.0T,20.0T,2.0M,22060.0,28.127 MW,281.276 TW
3,10.0T,200.0T,2.0M,220608.0,281.275 MW,28127.585 TW
4,100.0T,2000.0T,2.0M,2206085.0,2812.758 MW,2812758.526 TW
0,0.1T,2.0T,10.0M,11030.0,14.063 MW,2.813 TW
1,0.5T,10.0T,10.0M,55152.0,70.319 MW,70.319 TW
2,1.0T,20.0T,10.0M,110304.0,140.638 MW,281.276 TW
3,10.0T,200.0T,10.0M,1103042.0,1406.379 MW,28127.585 TW
4,100.0T,2000.0T,10.0M,11030425.0,14063.792 MW,2812758.526 TW
